  # Coleta de Dados Meteorológicos e de Geração de Energia Solar



  Este notebook documenta o processo detalhado de coleta e processamento dos dados meteorológicos fornecidos pelo INMET (Instituto Nacional de Meteorologia) e dos dados de geração de energia solar disponibilizados pela ONS (Operador Nacional do Sistema Elétrico). O objetivo é integrar essas fontes de dados em um único banco de dados para análise e previsão de geração de energia solar.

## Importação de Bibliotecas Necessárias

In [1]:
import os  # Biblioteca para manipulação de arquivos e diretórios
import sys  # Biblioteca para manipulação de caminhos do sistema
import pandas as pd  # Biblioteca para manipulação de dataframes

# Ajuste do sys.path para incluir o diretório src, onde estão os scripts específicos do projeto
current_dir = os.getcwd()  # Diretório de trabalho atual
src_path = os.path.abspath(os.path.join(current_dir, '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

# Importar funções específicas do projeto
from data_collection import *
from database import create_connection, query_data

## Conexão com o Banco de Dados


In [2]:
# Conexão com o banco de dados
current_dir = os.getcwd()  # Diretório de trabalho atual, que é dentro de 'data'
repository_dir = os.path.abspath(os.path.join(current_dir, '..'))  # Voltar um nível para o diretório do repositório
db_file = os.path.join(repository_dir, 'data', 'processed', 'dados_solar.db')
conn = create_connection(db_file)  # Função personalizada para criar uma conexão com o banco de dados

  ## Dados Meteorológicos

  ### Configurações Iniciais



  Nesta seção, definimos a URL base para os dados meteorológicos do INMET, os anos que queremos baixar e o diretório de destino onde os dados serão armazenados. Também estabelecemos a conexão com o banco de dados onde os dados processados serão armazenados.

In [ ]:
# Configurações
base_url = "https://portal.inmet.gov.br/uploads/dadoshistoricos/"  # URL base para download dos dados do INMET

# Lista de anos para baixar os dados
ano_inicial = 2003
ano_final = 2003

# Diretório de destino para armazenar os dados baixados
dest_dir = os.path.join(repository_dir, 'data', 'raw', 'INMET')

  ### Baixar e Extrair Arquivos ZIP do INMET



  Utilizamos a função `baixar_e_extrair_zip` para baixar e extrair os arquivos ZIP do INMET. Esta função automatiza o processo de download e extração, facilitando a coleta de dados para múltiplos anos.

In [ ]:
print(f"Importando dados do INMET de {ano_inicial} a {ano_final}...")

# Baixar e extrair dados
for ano in range(ano_inicial, ano_final + 1):
    url = base_url + f"{ano}.zip"
    baixar_e_extrair_zip(url, dest_dir)

  ### Processar e Armazenar Dados do INMET



  Nesta etapa, percorremos o diretório onde os dados foram extraídos, lemos os arquivos CSV relevantes, processamos e armazenamos os dados no banco de dados. Os arquivos CSV que começam com "INMET_SE_MG" são especificamente selecionados para este processamento.

In [ ]:
# Processar e armazenar dados do INMET
for root, dirs, files in os.walk(dest_dir):  # Percorre todas as pastas e arquivos no diretório de destino
    for file in files:
        if file.startswith("INMET_SE_MG"):  # Seleciona arquivos que começam com "INMET_SE_MG"
            file_path = os.path.join(root, file)  # Caminho completo do arquivo
            df = ler_csv_com_metadados(file_path)  # Lê o arquivo CSV com metadados
            df, colunas_importantes = filtrar_colunas(df)  # Filtra as colunas importantes
            df = renomear_colunas(df, colunas_importantes)  # Renomeia as colunas para um padrão consistente
            df.to_sql('dados_inmet', conn, if_exists='append', index=False)  # Armazena os dados no banco de dados

  ## Dados de Geração de Energia Solar (ONS)

  ### Configurações Iniciais



  Nesta seção, definimos os anos que queremos baixar e o diretório de destino onde os dados serão armazenados.

In [ ]:
# Configurações
ano_inicial = 2013
ano_final = 2023

# Diretório de destino para armazenar os dados baixados
dest_dir = os.path.join(repository_dir, 'data', 'raw', 'ONS')

  ### Coleta, Processamento e Armazenamento de Dados da ONS

Utilizamos funções definidas para coletar e processar os dados de geração de energia solar da ONS. Os dados são coletados tanto em formato CSV quanto em formato Excel, dependendo do ano.

In [ ]:
# Coletar e processar dados da ONS
df_geracao = processar_dados_geracao(ano_inicial, ano_final, dest_dir)

# Salvar dados no banco de dados
df_geracao['din_instante'] = pd.to_datetime(df_geracao['din_instante'])

# Supondo que você tem outro DataFrame df_geracao que também contém Timestamps
df_geracao['din_instante'] = df_geracao['din_instante'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Salvar dados no banco de dados
df_geracao.to_sql('dados_usinas', conn, if_exists='replace', index=False)

### Relação Usina - CEG

In [3]:
# Caminho para carregar os dados da ONS de usinas por CEG
arq_path = os.path.join(repository_dir, 'data', 'external', 'Tabela Relação Usinas (ONS).xlsx')

# Carregar os dados da ONS de usinas por CEG
df_ons = pd.read_excel(arq_path)

# Salvar dados no banco de dados
df_ons.to_sql('dados_ceg', conn, if_exists='replace', index=False)

# Fechar a conexão com o banco de dados
print("Dados importados com sucesso!")
conn.close()


113